In [1]:
%load_ext autoreload
%autoreload 2
from simuq import QSystem
from simuq import Qubit
from simuq.ibm import IBMProvider
import numpy as np
ibm = IBMProvider(
    from_file="../../../qiskit_APIKEY", hub="ibm-q-ornl", group="ornl", project="phy147"
)

def GenQS(n=12, p=3):
    qs = QSystem()
    ql = [Qubit(qs) for i in range(n)]
    link = [(i, (i + 1) % n) for i in range(n)]
    if p==3:    
        parameter_list = np.array(
        [
            0.5702193 * 2,
            -0.58631086,
            0.85160685 * 2,
            -1.7058538,
            0.29468536 * 2,
            -1.132814,
        ]
    )/2
    if p==2:
        parameter_list = np.array([-3.8034,1.2438, -1.2467,-2.4899])/2
    if p==1:
        parameter_list = np.array([-0.7854,-2.3562])/2

    for i in range(p):
       
        h = 0
        for q0, q1 in link:
            h += parameter_list[2 * i] * ql[q0].Z * ql[q1].Z
        qs.add_evolution(h, 1)
        h = 0
        for q0 in range(n):
            h += parameter_list[2 * i+1] * ql[q0].X
        qs.add_evolution(h, 1)
    return qs

def calc_cut(count):
    cut=0
    n_shots=sum(count.values())
    for key in count:
        cut+=count[key]*calc_weight(key)/n_shots
    return cut
def calc_weight(key):
    cut=0
    for i in range(len(key)):
        if key[i]!= key[(i+1)%len(key)]:
            cut+=1
    return cut

In [2]:
from qiskit import QuantumCircuit

all_h_circ = QuantumCircuit(12)
for i in range(12):
    all_h_circ.h(i)
for p in [1, 2, 3]:
    qs = GenQS(12, p)
    ibm.compile(qs, backend="ibmq_guadalupe", trotter_num=4, use_pulse=False, state_prep=all_h_circ)
    ibm.run(on_simulator=True)
    result = ibm.results(on_simulator=True)
    print(result)
    print(calc_cut(result))

{'100010010010': 0.000244140625, '010001101011': 0.000244140625, '101101010000': 0.000244140625, '011101001001': 0.000244140625, '001111111011': 0.000244140625, '100110101110': 0.000244140625, '001011100101': 0.000244140625, '101011001011': 0.000244140625, '011001010100': 0.000244140625, '110010100010': 0.000244140625, '010100011101': 0.000244140625, '101010100000': 0.000244140625, '010111110010': 0.000244140625, '000010100101': 0.000244140625, '101110011001': 0.000244140625, '000101001001': 0.000244140625, '101011011110': 0.000244140625, '100100101100': 0.000244140625, '001001110101': 0.000244140625, '001110110101': 0.000244140625, '111011010110': 0.000244140625, '100011101100': 0.000244140625, '110011001110': 0.000244140625, '010011001100': 0.000244140625, '111011101100': 0.000244140625, '011000110101': 0.00048828125, '111001011010': 0.000244140625, '101000011110': 0.000244140625, '001010010001': 0.000244140625, '001010011000': 0.000244140625, '111011010100': 0.000244140625, '0110110